# Regression Problem

### 1. Import necessary libraries:

In [14]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer

from mltools.regression_tools import LinearRegressor
from mltools import regression_tools as RT

### 2. Load data as a Pandas DataFrame:

In [2]:
path_irrad = "../data/G07A_DATOS_IRRAD.csv"
df_orig_irrad = pd.read_csv(path_irrad)

# Parse the date (string) for it to be a datetime with format yyyy/mm/dd
df_orig_irrad['FECHA'] = pd.to_datetime(df_orig_irrad['FECHA'], format='%Y-%m-%d')

s = df_orig_irrad.loc[:,'FECHA']
df_orig_irrad['FECHA'] =  s.dt.date

In [3]:
path_util = "../data/G07A_DATOS_UTIL.csv"
df_orig_util = pd.read_csv(path_util)

# Parse the date (string) for it to be a datetime with format yyyy/mm/dd
df_orig_util['FECHA'] = pd.to_datetime(df_orig_util['FECHA'], format='%Y-%m-%d')

s = df_orig_util.loc[:,'FECHA']
df_orig_util['FECHA'] =  s.dt.date

In [4]:
df = pd.merge(df_orig_irrad, df_orig_util, on=['FECHA', 'ANNO', 'MES', 'DIA', 'DIASEM'] , how='inner')
df.head()

,FECHA,IRRADH00,IRRADH03,IRRADH06,IRRADH09,IRRADH12,IRRADH15,IRRADH18,IRRADH21,ANNO,...,DIA,DIASEM,UTILH00,UTILH03,UTILH06,UTILH09,UTILH12,UTILH15,UTILH18,UTILH21
0,2015-01-01,0.0,0.0,0.0,414536.22,1193085.20,481816.38,0.0,0.0,2015,...,1,3,0.0,0.0,0.027848,0.235443,0.263291,0.039241,0.0,0.0
1,2015-01-02,0.0,0.0,0.0,319990.20,926238.10,182854.10,0.0,0.0,2015,...,2,4,0.0,0.0,0.027848,0.181857,0.205063,0.024051,0.0,0.0
2,2015-01-03,0.0,0.0,0.0,403464.62,1146347.20,360073.94,0.0,0.0,2015,...,3,5,0.0,0.0,0.029536,0.237975,0.235865,0.036287,0.0,0.0
3,2015-01-04,0.0,0.0,0.0,349597.03,1016177.06,315783.62,0.0,0.0,2015,...,4,6,0.0,0.0,0.029114,0.196203,0.210970,0.026160,0.0,0.0
4,2015-01-05,0.0,0.0,0.0,408705.38,1090662.90,532075.10,0.0,0.0,2015,...,5,0,0.0,0.0,0.028270,0.205485,0.228270,0.036287,0.0,0.0


### 3. Split the data into training and test sets:

In [5]:
# Define input and output matrices:
INPUTS = ['IRRADH00', 'IRRADH03', 'IRRADH06', 'IRRADH09', 'IRRADH12',
          'IRRADH15', 'IRRADH18', 'IRRADH21', 'ANNO', 'MES', 'DIA', 'DIASEM']
OUTPUTS = ['UTILH00', 'UTILH03', 'UTILH06', 'UTILH09', 
          'UTILH12', 'UTILH15', 'UTILH18', 'UTILH21']

X = df[INPUTS]
Y = df[OUTPUTS]

# Split:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y,
                                                    test_size=0.3,  # Percentage of test data
                                                    random_state=0) # Seed for replication

In [6]:
# Create dataset to store model predictions:
dfTR_eval = X_train.copy()
dfTR_eval[OUTPUTS] = Y_train

dfTS_eval = X_test.copy()
dfTS_eval[OUTPUTS] = Y_test

### 4. Training of Linear Regression Models:

As it has been indicated that the solar utilization depends on irradiation from a given time interval, we will be considering separate linear regression models for each hour.

In [10]:
hours = ['00', '03', '06', '09', '12', '15', '18', '21']

# Initialize a dictionary to store the trained models for each hour
# and a second one to store the Cross-Validation scores for each trained model:
hourly_models = {}
CV_score_hourly = {}

# Train a linear regression model for each hour:
for hour in hours:

    # Define input and output features for the current hour:
    inputs_hourly = ['IRRADH' + hour, 'ANNO', 'MES', 'DIA', 'DIASEM']
    output_hourly = 'UTILH' + hour
    
    # Create input and output matrices for the current hour:
    X_train_hourly = X_train[inputs_hourly]
    y_train_hourly = Y_train[output_hourly]
    
    # ----- 
    # -----

    # Train a linear regression model for the current hour:
    INPUTS_LR = inputs_hourly

    # Preparation of the numeric variables by scaling:
    numeric_transformer = Pipeline(steps=[('scaler', StandardScaler())])

    # Creation of a preprocessor to perform the steps defined above:
    preprocessor = ColumnTransformer(transformers=[('num', numeric_transformer, INPUTS_LR)])

    # Establishment the Linear Regression Model within a Pipeline:
    LinReg_fit_h = Pipeline(steps=[('Prep',preprocessor), 
                        ('LinReg',LinearRegressor(fit_intercept=True))]) 

    # Train model:
    LinReg_fit_h.fit(X_train_hourly, y_train_hourly);
    
    # Store the trained model in the dictionary:
    hourly_models[hour] = LinReg_fit_h

    # ----- 
    # -----

    # Store Cross-Validation score:
    CV_score_h = cross_val_score(LinReg_fit_h, X_train_hourly, y_train_hourly, cv=10, scoring='neg_root_mean_squared_error').mean()
    CV_score_hourly[hour] = CV_score_h

In [13]:
print("CV negative RMSE per time interval is:\n")
for key, value in CV_score_hourly.items():
    print(f"{key}: {value}")

CV negative RMSE per time interval is:

00: -0.0016302126706607086
03: -0.006360087538595756
06: -0.03768178725968737
09: -0.07104960836432125
12: -0.07232097189778289
15: -0.036281663767392766
18: -0.0035655914772534866
21: -0.0013063064756179488
